In [1]:
# Import Splinter, BeautifulSoup, pandas, time and webdriver manager
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time as time
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#Set executable path
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# MARS NEWS

In [3]:
#Got to NASA's Mars news website
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
#convert browser into a soup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
#print(soup.prettify())

In [6]:
#Go through HTML contents of page, find top-listed item

In [7]:
top_item = soup.select_one('ul.item_list ')
header=top_item.find("div", class_= "content_title")
header=header.text.strip()
header

"NASA's New Mars Rover Will Use X-Rays to Hunt Fossils"

In [8]:
#Find article contents associated with first 

In [9]:
par_text=top_item.find("div", class_= "article_teaser_body")
par_text=par_text.text.strip()
par_text

"PIXL, an instrument on the end of the Perseverance rover's arm, will search for chemical fingerprints left by ancient microbes."

# MARS Image search

In [10]:
#Got to NASA's Mars news website
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [11]:
se = browser.find_by_id('full_image')
se.click()

In [12]:
#Click the button that says "more info", by looking for xml path associated with "more info" button

In [13]:
look_for_text = '//*[@id="fancybox-lock"]/div/div[2]/div/div[1]/a[2]'

matching_elements = browser.find_by_xpath(look_for_text)

In [14]:
matching_elements.click()

In [15]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [16]:
#We know that the mian image that we want is of the image class, "main image"

In [17]:
images = soup.find_all("img", { "class" : "main_image" })

In [18]:
#Let's check and make sure we're finding the correct image
print(images)

[<img alt="Beam Wave Guide antennas located at Goldstone, CA, known as the 'Beam Waveguide Cluster'. " class="main_image" src="/spaceimages/images/largesize/PIA17793_hires.jpg" title="Beam Wave Guide antennas located at Goldstone, CA, known as the 'Beam Waveguide Cluster'. "/>]


In [19]:
for image in images:
    image_href = f"{image['src']}"

In [20]:
print(image_href)

/spaceimages/images/largesize/PIA17793_hires.jpg


In [21]:
#We have our stem url, and we also have the src that takes us to the large scale image. Combine into a single link.
stem_url = 'https://www.jpl.nasa.gov'
featured_image_url=stem_url+image_href
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17793_hires.jpg


# MARS Facts

In [22]:
import pandas as pd
url = 'https://space-facts.com/mars/'

In [23]:
df = pd.read_html(url)
for df in df:
    try:
        df = df.rename(columns={0:"Fact", 1:"Data"})
        df = df.set_index("Fact")
        #marsfacts_html = df.to_html().replace('\n', '')
        print(df)
        break
    except:
        continue


                                               Data
Fact                                               
Equatorial Diameter:                       6,792 km
Polar Diameter:                            6,752 km
Mass:                 6.39 × 10^23 kg (0.11 Earths)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.38 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                   -87 to -5 °C
First Record:                     2nd millennium BC
Recorded By:                   Egyptian astronomers


In [24]:
df.to_html('Mars_table.html')

# MARS Hemispheres

In [25]:
url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
stem_url= 'https://astrogeology.usgs.gov'

In [26]:
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


In [27]:
pages = soup.find_all('div', class_='item')

links = []
titles = []
for page in pages:
    links.append(stem_url + page.find('a')['href'])
    titles.append(page.find('h3').text.strip())
print(links)
titles


['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']


['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [28]:
browser.visit(links[0])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
finalurl = stem_url+soup.find('img',class_='wide-image')['src']
finalurl

'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'

In [29]:
img_url = []
for finalurl in links:
    browser.visit(finalurl)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    finalurl = stem_url+soup.find('img',class_='wide-image')['src']
    img_url.append(finalurl)
    
img_url

['https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [30]:
hemisphere_image_urls = []

for i in range(len(titles)):
    hemisphere_image_urls.append({'title':titles[i],'img_url':img_url[i]})

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]